In [1]:
import pickle
import json
import numpy as np

In [2]:
from transformers import AutoTokenizer

In [3]:
original_knowns = json.load(open("/work/pi_dhruveshpate_umass_edu/rseetharaman_umass_edu/repo-for-paper/attention-contributions/datasets/known_1000.json"))

In [4]:
subject_mapping = dict()
for o in original_knowns:
    subject_mapping[o['known_id']] = o['subject']

In [5]:
data_sample = pickle.load(open("/work/pi_dhruveshpate_umass_edu/rseetharaman_umass_edu/repo-for-paper/attention-contributions/outputs/llama-2-base/run-rag-synthetic-position-0/data/stats_0.pkl", "rb"))

In [45]:
def find_sub_list(sl,l, offset=0):
    sll=len(sl)
    indices = []
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if ind < offset:
            continue
        if l[ind:ind+sll]==sl:
            indices.append((ind,ind+sll))
    return indices
        
def find_within_text(prompt, substring, tokenizer):
    """
    A function that identifies the indices of tokens of a part of the prompt. 
    By default we use the first occurence. 
    """
    prompt_tokens = tokenizer.encode(prompt)
    # Use this one if using LLama-2. otherwise use the following line.
    part_tokens = tokenizer.encode(substring)[2:]
    # Use this one for Phi-2. 
    # This is due to weird differences in tokenizers of the models.
    # part_tokens = [tokenizer.encode(p) for p in parts]
    part_token_indices = find_sub_list(part_tokens, prompt_tokens)
    return part_token_indices

In [11]:
tokenizer = AutoTokenizer.from_pretrained("/work/pi_dhruveshpate_umass_edu/rseetharaman_umass_edu/llama-2-7b-hf")

In [7]:
data_sample.keys()

dict_keys(['completion', 'full_prompt', 'num_prompt_tokens', 'prompt_tokens', 'prompt', 'constraints', 'token_labels', 'all_max_attention_weights', 'all_token_contrib_norms', 'token_contrib_norms_constraints', 'attention_weights_constraints', 'id'])

In [8]:
subject_mapping[0]

'Vinson Massif'

In [53]:
all_subject_positions = find_within_text(data_sample['prompt'], f' {subject_mapping[0]}', tokenizer)

In [50]:
input_ids = tokenizer(data_sample['prompt'])['input_ids']

In [54]:
for (s,e) in all_subject_positions:
    assert(tokenizer.decode(input_ids[s:e]) == subject_mapping[0])

In [59]:
s,e = all_subject_positions[-1]

In [60]:
data_sample['all_token_contrib_norms'].shape

(32, 325)

In [66]:
np.sum(data_sample['all_token_contrib_norms'][:,s:e], axis=1).shape

(32,)

In [67]:
data_sample['all_max_attention_weights'].shape

(32, 325)

In [12]:
len(data_sample['token_contrib_norms_constraints'])

2

In [13]:
data_sample['token_contrib_norms_constraints'][0].shape

(32, 32, 1)

In [14]:
data_sample['token_contrib_norms_constraints'][1].shape

(32, 32, 4)